In [86]:
from torch_geometric.datasets import Planetoid

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch_geometric.nn import Linear, GATv2Conv
import torch.nn.functional as F

In [87]:
ds = Planetoid(root="/tmp/Cora", name="Cora")[0]

In [95]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):

        super().__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads, edge_dim=3, dropout=0.2)
        self.conv2 = GATv2Conv(hidden_channels * heads, hidden_channels, heads, edge_dim=3, dropout=0.2)
        self.lin1 = Linear(hidden_channels * heads, 32)
        self.lin2 = Linear(32, 8)
        self.linout = Linear(8, out_channels)

    def forward(self, x, edge_index, edge_attr=None):
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.linout(x)
        x = F.log_softmax(x)

        return x

In [89]:
model = GAT(
    in_channels=ds.x.shape[1],
    hidden_channels=32,
    heads=4,
    out_channels=7
)

sum(p.numel() for p in model.parameters() if p.requires_grad)

405863

In [90]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-3)

model.train()
for epoch in tqdm(range(10000), total=10000):
    optimizer.zero_grad()
    out = model(x=ds.x, edge_index=ds.edge_index)
    loss = F.nll_loss(out[ds.train_mask], ds.y[ds.train_mask])
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/10000 [00:00<?, ?it/s]

/var/folders/48/7kkqmll9029fmgt__8pzt5th0000gn/T/ipykernel_21653/681760877.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


tensor(1.9741, grad_fn=<NllLossBackward0>)
tensor(1.9740, grad_fn=<NllLossBackward0>)
tensor(1.9728, grad_fn=<NllLossBackward0>)
tensor(1.9711, grad_fn=<NllLossBackward0>)
tensor(1.9678, grad_fn=<NllLossBackward0>)
tensor(1.9707, grad_fn=<NllLossBackward0>)
tensor(1.9724, grad_fn=<NllLossBackward0>)
tensor(1.9647, grad_fn=<NllLossBackward0>)
tensor(1.9648, grad_fn=<NllLossBackward0>)
tensor(1.9647, grad_fn=<NllLossBackward0>)
tensor(1.9644, grad_fn=<NllLossBackward0>)
tensor(1.9625, grad_fn=<NllLossBackward0>)
tensor(1.9620, grad_fn=<NllLossBackward0>)
tensor(1.9629, grad_fn=<NllLossBackward0>)
tensor(1.9556, grad_fn=<NllLossBackward0>)
tensor(1.9618, grad_fn=<NllLossBackward0>)
tensor(1.9534, grad_fn=<NllLossBackward0>)
tensor(1.9550, grad_fn=<NllLossBackward0>)
tensor(1.9512, grad_fn=<NllLossBackward0>)
tensor(1.9449, grad_fn=<NllLossBackward0>)
tensor(1.9472, grad_fn=<NllLossBackward0>)
tensor(1.9443, grad_fn=<NllLossBackward0>)
tensor(1.9485, grad_fn=<NllLossBackward0>)
tensor(1.94

KeyboardInterrupt: 

In [91]:
model.eval()
pred = model(x=ds.x, edge_index=ds.edge_index)
pred_proba = torch.exp(pred[ds.test_mask])
pred = torch.argmax(pred_proba, axis=1)

/var/folders/48/7kkqmll9029fmgt__8pzt5th0000gn/T/ipykernel_21653/681760877.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In [94]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(ds.y[ds.test_mask], pred))
confusion_matrix(ds.y[ds.test_mask], pred)

0.656


array([[ 94,   4,   5,   7,   9,   2,   9],
       [  4,  64,   1,   4,  10,   7,   1],
       [  8,   1, 119,   3,  13,   0,   0],
       [ 48,  30,  14, 162,  37,  11,  17],
       [ 28,   0,  12,   3, 102,   3,   1],
       [  8,   3,   1,   2,   7,  74,   8],
       [ 14,   0,   0,   4,   3,   2,  41]])